## Write Python code to build a maze and train an agent to solve it. Show visualizations and progress.

### Step 1: Maze Environment & Visualization

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

# Maze configuration (0 = free space, 1 = wall)
maze = np.array([
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
])

start = (0, 0)
goal = (4, 4)
actions = ['up', 'down', 'left', 'right']
action_map = {
    'up': (-1, 0),
    'down': (1, 0),
    'left': (0, -1),
    'right': (0, 1)
}

### Step 2: Environment & Q-learning Setup

In [2]:
class MazeEnv:
    def __init__(self, maze, start, goal):
        self.maze = maze
        self.start = start
        self.goal = goal
        self.reset()

    def reset(self):
        self.agent_pos = self.start
        return self.agent_pos

    def step(self, action):
        move = action_map[action]
        new_pos = (self.agent_pos[0] + move[0], self.agent_pos[1] + move[1])
        if (0 <= new_pos[0] < self.maze.shape[0] and
            0 <= new_pos[1] < self.maze.shape[1] and
            self.maze[new_pos] == 0):
            self.agent_pos = new_pos

        reward = 1 if self.agent_pos == self.goal else -0.01
        done = self.agent_pos == self.goal
        return self.agent_pos, reward, done

### Step 3: Q-Learning Agent

In [3]:
q_table = {}
alpha = 0.1
gamma = 0.9
epsilon = 0.2
episodes = 500

env = MazeEnv(maze, start, goal)
path_history = []

def get_state_key(state):
    return str(state)

for ep in range(episodes):
    state = env.reset()
    episode_path = [state]

    for step in range(100):
        state_key = get_state_key(state)
        if state_key not in q_table:
            q_table[state_key] = {a: 0 for a in actions}

        # Epsilon-greedy action selection
        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = max(q_table[state_key], key=q_table[state_key].get)

        next_state, reward, done = env.step(action)
        next_state_key = get_state_key(next_state)

        if next_state_key not in q_table:
            q_table[next_state_key] = {a: 0 for a in actions}

        # Q-value update
        q_table[state_key][action] += alpha * (reward + gamma * max(q_table[next_state_key].values()) - q_table[state_key][action])

        state = next_state
        episode_path.append(state)

        if done:
            break

    path_history.append(episode_path)

### Step 4: Visualization of Progress

In [4]:
def draw_maze_with_path(maze, path, title="Maze"):
    plt.imshow(maze, cmap='binary')
    path_x, path_y = zip(*path)
    plt.plot(path_y, path_x, marker='o', color='blue')
    plt.scatter(start[1], start[0], c='green', s=100, label='Start')
    plt.scatter(goal[1], goal[0], c='red', s=100, label='Goal')
    plt.legend()
    plt.title(title)
    plt.axis('off')

# Show last episode path
draw_maze_with_path(maze, path_history[-1], "Final Episode Path")
plt.show()

### (Optional) Animation of Training Progress

In [5]:
fig = plt.figure()

def update(i):
    plt.clf()
    draw_maze_with_path(maze, path_history[i], f"Episode {i + 1}")

ani = animation.FuncAnimation(fig, update, frames=range(0, episodes, 10), repeat=False)
plt.show()

<Figure size 640x480 with 0 Axes>

# Interactive Debugging

## Debug the "Animation of Training Progress" code

In [6]:
%matplotlib notebook

In [7]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Set up the figure
fig, ax = plt.subplots()

def update(i):
    ax.clear()
    draw_maze_with_path(maze, path_history[i], title=f"Episode {i + 1}")

# Interval = milliseconds between frames
ani = animation.FuncAnimation(fig, update, frames=range(0, len(path_history), 10), interval=200, repeat=False)

plt.show()

<IPython.core.display.Javascript object>

In [8]:
ani.save("training_progress.gif", writer="pillow")
# or
# ani.save("training_progress.mp4", writer="ffmpeg")

## Make the maze larger

### Step 1: Generate a Larger Solvable Maze (10x10)

We'll generate a random maze but ensure that there is a path from start to goal using a randomized depth-first search.

In [9]:
import numpy as np
import random

def generate_maze(rows=10, cols=10):
    maze = np.ones((rows, cols), dtype=int)  # Start with all walls
    visited = np.zeros_like(maze, dtype=bool)

    def dfs(r, c):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        random.shuffle(directions)
        visited[r, c] = True
        maze[r, c] = 0  # Mark path

        for dr, dc in directions:
            nr, nc = r + 2*dr, c + 2*dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                maze[r + dr, c + dc] = 0  # Remove wall between
                dfs(nr, nc)

    # Start DFS from top-left
    dfs(0, 0)
    maze[0, 0] = 0  # Ensure start is open
    maze[rows - 1, cols - 1] = 0  # Ensure goal is open
    return maze

maze = generate_maze(10, 10)
start = (0, 0)
goal = (9, 9)

### Step 2: Environment & Agent Code

Update the MazeEnv class and Q-learning logic from earlier — no change needed, just re-use them with the new maze, start, and goal.

### Step 3: Run Training

Keep everything from earlier (Q-table setup, training loop), just update episodes to, say, 1000 for better learning in the larger space.

In [10]:
q_table = {}
alpha = 0.1
gamma = 0.9
epsilon = 0.2
episodes = 500

env = MazeEnv(maze, start, goal)
path_history = []

def get_state_key(state):
    return str(state)

for ep in range(episodes):
    state = env.reset()
    episode_path = [state]

    for step in range(1000):
        state_key = get_state_key(state)
        if state_key not in q_table:
            q_table[state_key] = {a: 0 for a in actions}

        # Epsilon-greedy action selection
        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = max(q_table[state_key], key=q_table[state_key].get)

        next_state, reward, done = env.step(action)
        next_state_key = get_state_key(next_state)

        if next_state_key not in q_table:
            q_table[next_state_key] = {a: 0 for a in actions}

        # Q-value update
        q_table[state_key][action] += alpha * (reward + gamma * max(q_table[next_state_key].values()) - q_table[state_key][action])

        state = next_state
        episode_path.append(state)

        if done:
            break

    path_history.append(episode_path)

In [11]:
draw_maze_with_path(maze, path_history[-1], "Final Episode Path on 10x10 Maze")
plt.show()

In [12]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots()

def update(i):
    ax.clear()
    draw_maze_with_path(maze, path_history[i], title=f"Episode {i + 1}")

ani = animation.FuncAnimation(fig, update, frames=range(0, len(path_history), 25), interval=200, repeat=False)
plt.show()

<IPython.core.display.Javascript object>

In [13]:
ani.save("training_progress_larger.gif", writer="pillow")

# Optimize the algorithm for efficiency.

### 1. Replace String Keys with Tuples

In [14]:
# Use state as (row, col) tuple instead of str
def get_state_key(state):
    return state  # already a tuple

### 2. Use Integer Actions

In [15]:
actions = [0, 1, 2, 3]  # 0: up, 1: down, 2: left, 3: right
action_map = {
    0: (-1, 0),  # up
    1: (1, 0),   # down
    2: (0, -1),  # left
    3: (0, 1)    # right
}

### 3. Efficient Q-Learning Code (Optimized)

In [16]:
q_table = {}
initial_alpha = 0.5
gamma = 0.95
initial_epsilon = 0.9
min_epsilon = 0.05
epsilon_decay = 0.995
alpha_decay = 0.995
episodes = 1000

epsilon = initial_epsilon
alpha = initial_alpha

successful_episodes = 0

for ep in range(episodes):
    state = env.reset()
    episode_path = [state]

    for _ in range(1000):
        # Lazy init
        if state not in q_table:
            q_table[state] = np.zeros(len(actions))

        # Epsilon-greedy
        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = int(np.argmax(q_table[state]))

        move = action_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])

        if (0 <= next_state[0] < maze.shape[0] and
            0 <= next_state[1] < maze.shape[1] and
            maze[next_state] == 0):
            pass  # valid move
        else:
            next_state = state  # stay in place

        reward = 1 if next_state == goal else -0.01
        done = next_state == goal

        if next_state not in q_table:
            q_table[next_state] = np.zeros(len(actions))

        # Q update
        q_table[state][action] += alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        state = next_state
        episode_path.append(state)

        if done:
            successful_episodes += 1
            break

    path_history.append(episode_path)

    # Decay learning rate & epsilon
    epsilon = max(min_epsilon, epsilon * epsilon_decay)
    alpha = alpha * alpha_decay

    # Optional early stopping
    if successful_episodes >= 20 and len(set(path_history[-10])) == 1:
        print(f"Early stop at episode {ep}")
        break


### Visualization (unchanged)

In [17]:
draw_maze_with_path(maze, path_history[-1], "Final Episode Path on 10x10 Maze")
plt.show()

In [18]:
ani.save("training_progress_optimized.gif", writer="pillow")

In [19]:
from IPython.display import HTML
HTML(ani.to_jshtml())

# Show the agent's progress at each step. Use Visualizations

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

# Set up a 10x10 solvable maze
def generate_maze(rows=10, cols=10):
    maze = np.ones((rows, cols), dtype=int)  # Start with all walls
    visited = np.zeros_like(maze, dtype=bool)

    def dfs(r, c):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        random.shuffle(directions)
        visited[r, c] = True
        maze[r, c] = 0  # Mark path

        for dr, dc in directions:
            nr, nc = r + 2*dr, c + 2*dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                maze[r + dr, c + dc] = 0  # Remove wall between
                dfs(nr, nc)

    dfs(0, 0)
    maze[0, 0] = 0
    maze[rows - 1, cols - 1] = 0
    return maze

# Define the environment
class MazeEnv:
    def __init__(self, maze, start, goal):
        self.maze = maze
        self.start = start
        self.goal = goal
        self.reset()

    def reset(self):
        self.agent_pos = self.start
        return self.agent_pos

    def step(self, action):
        move = action_map[action]
        new_pos = (self.agent_pos[0] + move[0], self.agent_pos[1] + move[1])
        if (0 <= new_pos[0] < self.maze.shape[0] and
            0 <= new_pos[1] < self.maze.shape[1] and
            self.maze[new_pos] == 0):
            self.agent_pos = new_pos
        reward = 1 if self.agent_pos == self.goal else -0.01
        done = self.agent_pos == self.goal
        return self.agent_pos, reward, done

# Drawing function
def draw_maze_with_path(ax, maze, path, title="Maze"):
    ax.imshow(maze, cmap='binary')
    path_x, path_y = zip(*path)
    ax.plot(path_y, path_x, marker='o', color='blue')
    ax.scatter(start[1], start[0], c='green', s=100, label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=100, label='Goal')
    ax.set_title(title)
    ax.axis('off')

# Maze setup
maze = generate_maze(10, 10)
start = (0, 0)
goal = (9, 9)
actions = [0, 1, 2, 3]  # 0: up, 1: down, 2: left, 3: right
action_map = {
    0: (-1, 0),
    1: (1, 0),
    2: (0, -1),
    3: (0, 1)
}

# Initialize environment
env = MazeEnv(maze, start, goal)

# Q-learning parameters
q_table = {}
alpha = 0.5
gamma = 0.95
epsilon = 0.9
epsilon_decay = 0.995
min_epsilon = 0.05
episodes = 300

path_history = []

# Train agent and store paths
for ep in range(episodes):
    state = env.reset()
    episode_path = [state]
    for _ in range(100):
        if state not in q_table:
            q_table[state] = np.zeros(len(actions))

        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = int(np.argmax(q_table[state]))

        move = action_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        if (0 <= next_state[0] < maze.shape[0] and
            0 <= next_state[1] < maze.shape[1] and
            maze[next_state] == 0):
            pass
        else:
            next_state = state

        reward = 1 if next_state == goal else -0.01
        done = next_state == goal

        if next_state not in q_table:
            q_table[next_state] = np.zeros(len(actions))

        q_table[state][action] += alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        state = next_state
        episode_path.append(state)

        if done:
            break

    path_history.append(episode_path)
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

# Animation setup
fig, ax = plt.subplots()

def update(i):
    ax.clear()
    draw_maze_with_path(ax, maze, path_history[i], f"Episode {i+1}")

from IPython.display import HTML

ani = animation.FuncAnimation(fig, update, frames=range(0, len(path_history), 10), interval=300, repeat=False)
HTML(ani.to_jshtml())

# ani

<IPython.core.display.Javascript object>

In [21]:
ani.save("training_progress_visualization.gif", writer="pillow")

In [22]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random

# Set up a 10x10 solvable maze
def generate_maze(rows=10, cols=10):
    maze = np.ones((rows, cols), dtype=int)  # Start with all walls
    visited = np.zeros_like(maze, dtype=bool)

    def dfs(r, c):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        random.shuffle(directions)
        visited[r, c] = True
        maze[r, c] = 0  # Mark path

        for dr, dc in directions:
            nr, nc = r + 2*dr, c + 2*dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                maze[r + dr, c + dc] = 0  # Remove wall between
                dfs(nr, nc)

    dfs(0, 0)
    maze[0, 0] = 0
    maze[rows - 1, cols - 1] = 0
    return maze



# Define the environment
class MazeEnv:
    def __init__(self, maze, start, goal):
        self.maze = maze
        self.start = start
        self.goal = goal
        self.reset()

    def reset(self):
        self.agent_pos = self.start
        return self.agent_pos

    def step(self, action):
        move = action_map[action]
        new_pos = (self.agent_pos[0] + move[0], self.agent_pos[1] + move[1])
        if (0 <= new_pos[0] < self.maze.shape[0] and
            0 <= new_pos[1] < self.maze.shape[1] and
            self.maze[new_pos] == 0):
            self.agent_pos = new_pos
        reward = 1 if self.agent_pos == self.goal else -0.01
        done = self.agent_pos == self.goal
        return self.agent_pos, reward, done

# Drawing function
def draw_maze_with_path(ax, maze, path, title="Maze"):
    ax.imshow(maze, cmap='binary')
    path_x, path_y = zip(*path)
    ax.plot(path_y, path_x, marker='o', color='blue')
    ax.scatter(start[1], start[0], c='green', s=100, label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=100, label='Goal')
    ax.set_title(title)
    ax.axis('off')

# Maze setup
maze = generate_maze(10, 10)
start = (0, 0)
goal = (9, 9)
actions = [0, 1, 2, 3]  # 0: up, 1: down, 2: left, 3: right
action_map = {
    0: (-1, 0),
    1: (1, 0),
    2: (0, -1),
    3: (0, 1)
}

# Initialize environment
env = MazeEnv(maze, start, goal)

# Q-learning parameters
q_table = {}
alpha = 0.5
gamma = 0.95
epsilon = 0.9
epsilon_decay = 0.995
min_epsilon = 0.05
episodes = 300

path_history = []

# Train agent and store paths
for ep in range(episodes):
    state = env.reset()
    episode_path = [state]
    for _ in range(100):
        if state not in q_table:
            q_table[state] = np.zeros(len(actions))

        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = int(np.argmax(q_table[state]))

        move = action_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        if (0 <= next_state[0] < maze.shape[0] and
            0 <= next_state[1] < maze.shape[1] and
            maze[next_state] == 0):
            pass
        else:
            next_state = state

        reward = 1 if next_state == goal else -0.01
        done = next_state == goal

        if next_state not in q_table:
            q_table[next_state] = np.zeros(len(actions))

        q_table[state][action] += alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        state = next_state
        episode_path.append(state)

        if done:
            break

    path_history.append(episode_path)
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

# Animation setup
fig, ax = plt.subplots()

def update(i):
    ax.clear()
    draw_maze_with_path(ax, maze, path_history[i], f"Episode {i+1}")

from IPython.display import HTML

ani = animation.FuncAnimation(fig, update, frames=range(0, len(path_history), 10), interval=300, repeat=False)
HTML(ani.to_jshtml())

# ani

<IPython.core.display.Javascript object>

# Debug Maze Generation

### Version 1

In [23]:
import numpy as np
import random

def generate_maze(rows=10, cols=10):
    maze = np.ones((rows, cols), dtype=int)
    visited = np.zeros_like(maze, dtype=bool)

    def dfs(r, c):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        random.shuffle(directions)
        visited[r, c] = True
        maze[r, c] = 0
        for dr, dc in directions:
            nr, nc = r + 2*dr, c + 2*dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                maze[r + dr, c + dc] = 0
                dfs(nr, nc)

    # Start DFS from top-left
    dfs(0, 0)

    start = (0, 0)
    goal = (rows - 1, cols - 1)

    # Ensure goal and at least one neighbor is open
    def make_goal_reachable(maze, goal):
        r, c = goal
        maze[r, c] = 0  # open goal cell
        neighbors = [(-1,0), (1,0), (0,-1), (0,1)]
        for dr, dc in neighbors:
            nr, nc = r + dr, c + dc
            if 0 <= nr < maze.shape[0] and 0 <= nc < maze.shape[1]:
                maze[nr, nc] = 0  # open neighbor path

    make_goal_reachable(maze, goal)
    return maze, start, goal

In [24]:
maze, start, goal = generate_maze(10, 10)

## Version 2

In [25]:
def generate_maze(rows=10, cols=10, start=(0, 0), goal=None):
    if goal is None:
        goal = (rows - 1, cols - 1)

    maze = np.ones((rows, cols), dtype=int)  # Start with all walls
    visited = np.zeros_like(maze, dtype=bool)

    def dfs(r, c):
        directions = [(0,1), (1,0), (0,-1), (-1,0)]
        random.shuffle(directions)
        visited[r, c] = True
        maze[r, c] = 0  # Mark path

        for dr, dc in directions:
            nr, nc = r + 2*dr, c + 2*dc
            if 0 <= nr < rows and 0 <= nc < cols and not visited[nr, nc]:
                maze[r + dr, c + dc] = 0  # Remove wall between
                dfs(nr, nc)

    # Run DFS to carve a path from the start
    dfs(start[0], start[1])

    # Make sure start and goal are open
    maze[start] = 0
    maze[goal] = 0

    # Ensure at least one neighbor of the goal is open
    def make_goal_reachable():
        r, c = goal
        for dr, dc in [(-1,0), (1,0), (0,-1), (0,1)]:
            nr, nc = r + dr, c + dc
            if 0 <= nr < rows and 0 <= nc < cols:
                maze[nr, nc] = 0  # open path

    make_goal_reachable()

    return maze

In [26]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import random




# Define the environment
class MazeEnv:
    def __init__(self, maze, start, goal):
        self.maze = maze
        self.start = start
        self.goal = goal
        self.reset()

    def reset(self):
        self.agent_pos = self.start
        return self.agent_pos

    def step(self, action):
        move = action_map[action]
        new_pos = (self.agent_pos[0] + move[0], self.agent_pos[1] + move[1])
        if (0 <= new_pos[0] < self.maze.shape[0] and
            0 <= new_pos[1] < self.maze.shape[1] and
            self.maze[new_pos] == 0):
            self.agent_pos = new_pos
        reward = 1 if self.agent_pos == self.goal else -0.01
        done = self.agent_pos == self.goal
        return self.agent_pos, reward, done

# Drawing function
def draw_maze_with_path(ax, maze, path, title="Maze"):
    ax.imshow(maze, cmap='binary')
    path_x, path_y = zip(*path)
    ax.plot(path_y, path_x, marker='o', color='blue')
    ax.scatter(start[1], start[0], c='green', s=100, label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=100, label='Goal')
    ax.set_title(title)
    ax.axis('off')

# Maze setup
maze = generate_maze(10, 10)
start = (0, 0)
goal = (9, 9)
actions = [0, 1, 2, 3]  # 0: up, 1: down, 2: left, 3: right
action_map = {
    0: (-1, 0),
    1: (1, 0),
    2: (0, -1),
    3: (0, 1)
}

# Initialize environment
env = MazeEnv(maze, start, goal)

# Q-learning parameters
q_table = {}
alpha = 0.5
gamma = 0.95
epsilon = 0.9
epsilon_decay = 0.995
min_epsilon = 0.05
episodes = 300

path_history = []

# Train agent and store paths
for ep in range(episodes):
    state = env.reset()
    episode_path = [state]
    for _ in range(100):
        if state not in q_table:
            q_table[state] = np.zeros(len(actions))

        if random.random() < epsilon:
            action = random.choice(actions)
        else:
            action = int(np.argmax(q_table[state]))

        move = action_map[action]
        next_state = (state[0] + move[0], state[1] + move[1])
        if (0 <= next_state[0] < maze.shape[0] and
            0 <= next_state[1] < maze.shape[1] and
            maze[next_state] == 0):
            pass
        else:
            next_state = state

        reward = 1 if next_state == goal else -0.01
        done = next_state == goal

        if next_state not in q_table:
            q_table[next_state] = np.zeros(len(actions))

        q_table[state][action] += alpha * (
            reward + gamma * np.max(q_table[next_state]) - q_table[state][action])

        state = next_state
        episode_path.append(state)

        if done:
            break

    path_history.append(episode_path)
    epsilon = max(min_epsilon, epsilon * epsilon_decay)

# Animation setup
fig, ax = plt.subplots()

def update(i):
    ax.clear()
    draw_maze_with_path(ax, maze, path_history[i], f"Episode {i+1}")

from IPython.display import HTML

ani = animation.FuncAnimation(fig, update, frames=range(0, len(path_history), 10), interval=300, repeat=False)
HTML(ani.to_jshtml())

# ani

<IPython.core.display.Javascript object>

In [27]:
final_path = path_history[-1]
fig, ax = plt.subplots()

def draw_agent_progress(i):
    ax.clear()
    ax.imshow(maze, cmap='binary')
    ax.scatter(start[1], start[0], c='green', s=100, label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=100, label='Goal')

    if i > 0:
        x_path, y_path = zip(*final_path[:i+1])
        ax.plot(y_path, x_path, marker='o', color='blue')

    r, c = final_path[i]
    ax.scatter(c, r, c='yellow', s=100, edgecolors='black', label='Agent')
    ax.set_title(f"Step {i+1} of {len(final_path)}")
    ax.axis('off')

agent_ani = animation.FuncAnimation(fig, draw_agent_progress, frames=len(final_path), interval=300, repeat=False)
plt.close()

agent_ani

<IPython.core.display.Javascript object>

AttributeError: 'NoneType' object has no attribute 'remove_callback'

In [28]:
final_path = path_history[-1]
fig, ax = plt.subplots()

def draw_agent_progress(i):
    ax.clear()
    ax.imshow(maze, cmap='binary')
    ax.scatter(start[1], start[0], c='green', s=100, label='Start')
    ax.scatter(goal[1], goal[0], c='red', s=100, label='Goal')

    if i > 0:
        x_path, y_path = zip(*final_path[:i+1])
        ax.plot(y_path, x_path, marker='o', color='blue')

    r, c = final_path[i]
    ax.scatter(c, r, c='yellow', s=100, edgecolors='black', label='Agent')
    ax.set_title(f"Step {i+1} of {len(final_path)}")
    ax.axis('off')

from IPython.display import HTML

agent_ani = animation.FuncAnimation(fig, draw_agent_progress, frames=len(final_path), interval=300, repeat=False)
HTML(agent_ani.to_jshtml())

<IPython.core.display.Javascript object>